In [1]:
from nltk.tokenize import word_tokenize,sent_tokenize
from nltk.corpus import stopwords

In [2]:
from collections import defaultdict

In [3]:
from string import punctuation

In [4]:
from heapq import nlargest

In [5]:
class FrequencySummarizer:
    def __init__(self,min_cut=0.1,max_cut=0.9):
        self._min_cut = min_cut
        self._max_cut = max_cut
        self._stopwords = set(stopwords.words('english')+list(punctuation))
    
    def _compute_frequencies(self,word_sent):
        freq = defaultdict(int)
        for sentence in word_sent:
            for words in sentence:
                if words not in self._stopwords:
                    freq[words] = freq[words] + 1
        max_freq = float(max(freq.values()))
        for word in freq.keys():
            freq[word] = freq[word]/max_freq
            if freq[word] >=self._max_cut or freq[word] <= self._min_cut:
                pass
        return freq
    
    def summarize(self,text,n):
        sents = sent_tokenize(text)
        assert n<= len(sents)
        word_sent = [word_tokenize(s.lower()) for s in sents]
        self._freq = self._compute_frequencies(word_sent)
        ranking = defaultdict(int)
        for i,sent in enumerate(word_sent):
            for word in sent:
                if word in self._freq:
                    ranking[i] += self._freq[word]
        sents_idx = nlargest(n,ranking,key=ranking.get)
        return [sents[j] for j in sents_idx]

    def _rank(self, ranking, n):
        return nlargest(n, ranking, key=ranking.get)

In [6]:
from urllib.request import urlopen
from bs4 import BeautifulSoup

In [7]:
def get_only_washingtpost_url(url):
    page = urlopen(url).read().decode('utf8')
    soup = BeautifulSoup(page)
    text = ' '.join(map(lambda p:p.text,soup.find_all('article')))
    soup2 = BeautifulSoup(text)
    text = ' '.join(map(lambda p:p.text,soup2.find_all('p')))
    return soup.title.text, text

In [8]:
someUrl  = 'https://www.washingtonpost.com/news/arts-and-entertainment/wp/2017/07/17/game-of-thrones-recap-a-return-to-dragonstone-and-a-question-of-alliances/?hpid=hp_no-name_hp-in-the-news%3Apage%2Fin-the-news&utm_term=.f476bd586446'
text = get_only_washingtpost_url(someUrl)

C:\Users\achu\Anaconda3\lib\site-packages\bs4\__init__.py:166: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

To get rid of this warning, change this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))


In [9]:
text

('‘Game of Thrones’ recap: A return to ‘Dragonstone’ and a question of alliances - The Washington Post',
 'The seventh season of HBO\'s "Game of Thrones" will premiere on July 16. (HBO)   Winter is here. The end game is coming. And watching “Game of Thrones” now feels different since we know that there are exactly 14 more episodes after tonight, and that everything must be resolved within that time frame. So when a full minute is spent on the Hound digging a grave for two random people from 20-some episodes ago, you want to yell, “Get on with it! We don’t have much time left!” So let’s just get to it. As always, please read Alyssa Rosenberg’s thoughtful review/analysis of each episode, which hopefully pairs well with this “let’s just try to figure out exactly what happened” recap. Arya Stark’s body count increases exponentially Collect your winnings if you thought Walder Frey would be the first voice to be heard in Season 7, since his throat-slitting at the hands of Arya Stark was one 

In [10]:
fs = FrequencySummarizer()

In [11]:
summary = fs.summarize(text[1],1)

In [12]:
summary

['Cersei wants to push forward to vanquish her many enemies; Jaime notes that those enemies surround them on all sides and that Cersei may like to say she’s the Queen of the Seven Kingdoms, but really she has “three at best.” Jaime is still processing the death of Tommen —\xa0“Our baby boy killed himself,” he pleads with her —\xa0while Cersei says of her children (all three of them dead now), “I loved them, I did, but they’re ashes now and we’re flesh and blood.']